In [1]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as pltv
import gower
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
df = pd.read_pickle("../data/processed.pkl")
categorical_columns = df.select_dtypes(include=['category']).columns

In [3]:
import pandas as pd
import prince

# Load the dataset
data = df

# Select the categorical columns for MCA
mca_columns = categorical_columns  # Replace with your categorical column names

# Create an instance of the MCA class
mca = prince.MCA(
    n_components=20,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)

# Fit the MCA model to the data
mca.fit(data[mca_columns])

MCA(n_components=20, n_iter=3, random_state=42)

In [4]:
row_coords = mca.row_coordinates(df)

In [11]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.preprocessing import LabelEncoder



# Combine the original categorical data with the cluster labels
clustered_data = data.copy()
clustered_data['Cluster'] = labels_optimal

for column in mca_columns:
    clustered_data[column] = clustered_data[column].astype(str)

# Create a label encoder for each categorical feature
label_encoders = {}
for column in mca_columns:
    label_encoders[column] = LabelEncoder()
    clustered_data[column] = label_encoders[column].fit_transform(clustered_data[column])

# Train a decision tree classifier
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(clustered_data[mca_columns], clustered_data['Cluster'])

# Print the decision tree rules
print("Decision Tree Rules:")
tree_rules = export_text(dt, feature_names=mca_columns)
print(tree_rules)

# Function to predict the cluster for a given data point
def predict_cluster(data_point):
    # Encode the categorical features of the data point
    encoded_data_point = data_point.copy()
    for column in mca_columns:
        encoded_data_point[column] = label_encoders[column].transform(encoded_data_point[column].values.reshape(-1))
    
    cluster = dt.predict(encoded_data_point[mca_columns].values.reshape(1, -1))[0]
    return cluster

# Example usage: Predict the cluster for a specific data point
example_data_point = clustered_data.iloc[0]

Decision Tree Rules:
|--- During the past 12 months have you received food stamps (FOODSTMP) <= 3.50
|   |--- Are you male or female? (CELLSEX1) <= 0.50
|   |   |--- How often do you think about your race? (RRCOGNT2) <= 8.50
|   |   |   |--- Asked in person or by form how much you drink? (ASBIDRNK) <= 3.50
|   |   |   |   |--- Live With a Problem Drinker/Alcoholic? (ACEDRINK) <= 3.50
|   |   |   |   |   |--- class: 27
|   |   |   |   |--- Live With a Problem Drinker/Alcoholic? (ACEDRINK) >  3.50
|   |   |   |   |   |--- class: 23
|   |   |   |--- Asked in person or by form how much you drink? (ASBIDRNK) >  3.50
|   |   |   |   |--- Did you vape marijuana or cannabis? (MARJVAPE) <= 3.50
|   |   |   |   |   |--- class: 14
|   |   |   |   |--- Did you vape marijuana or cannabis? (MARJVAPE) >  3.50
|   |   |   |   |   |--- class: 7
|   |   |--- How often do you think about your race? (RRCOGNT2) >  8.50
|   |   |   |--- How Often Did Anyone Ever Force You to Have Sex? (ACEHVSEX) <= 3.50
|  